# Sighash modifiers - BX Exercise
In this exercise, we spend two p2pkh inputs to two p2pkh outputs.
<br>The two inputs are signed with sighash modifiers `single|anyonecanpay` and `none`.
<br>The second output is not endorsed by any of the two signers and can be added after signing.

<br>
<img src="images/transaction_singleany_none.jpg" alt="drawing" style="" width="800px"/>


<hr style="border: 0.5px dashed #000;">

## 1. Restore wallet and generate spending key pairs.



In [ ]:
# my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="seven mail crash you unit small assume express wedding cloud work potato"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)

### Restore your spending keys.

We will be spending the funds controlled by these keys from account 1. 
* `m/44'/1'/1'/0/1`
* `m/44'/1'/1'/0/2`.


In [ ]:
# hd_m_44h_1h_1h_0_1=

# privatekey_44h_1h_1h_0_1=
# publickey_44h_1h_1h_0_1=
# publickeyhash_44h_1h_1h_0_1=
# address_44h_1h_1h_0_1=

hd_m_44h_1h_1h_0_1=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 1)

privatekey_44h_1h_1h_0_1=$(bx hd-to-ec $hd_m_44h_1h_1h_0_1)
publickey_44h_1h_1h_0_1=$(bx hd-to-public $hd_m_44h_1h_1h_0_1 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_1=$(bx sha256 $publickey_44h_1h_1h_0_1 | bx ripemd160)
address_44h_1h_1h_0_1=$(bx hd-to-public $hd_m_44h_1h_1h_0_1 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_1
echo $publickeyhash_44h_1h_1h_0_1

In [ ]:
# hd_m_44h_1h_1h_0_2=

# privatekey_44h_1h_1h_0_2=
# publickey_44h_1h_1h_0_2=
# publickeyhash_44h_1h_1h_0_2=
# address_44h_1h_1h_0_2=

hd_m_44h_1h_1h_0_2=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 2)

privatekey_44h_1h_1h_0_2=$(bx hd-to-ec $hd_m_44h_1h_1h_0_2)
publickey_44h_1h_1h_0_2=$(bx hd-to-public $hd_m_44h_1h_1h_0_2 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_2=$(bx sha256 $publickey_44h_1h_1h_0_2 | bx ripemd160)
address_44h_1h_1h_0_2=$(bx hd-to-public $hd_m_44h_1h_1h_0_2 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_2
echo $publickeyhash_44h_1h_1h_0_2

### Fetch previous UTXO's you are spending.

In [ ]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index_0]"
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index_1]"

bx fetch-tx 11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea -f json | jq ".transaction.outputs[1]"
bx fetch-tx 11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea -f json | jq ".transaction.outputs[2]"

In [ ]:
# previous_txid0=
# previous_output_index0=
# previous_output_amount0=

# previous_txid1=
# previous_output_index1=
# previous_output_amount1=

previous_txid0=11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea
previous_output_index0=1
previous_output_amount0=499550

previous_txid1=11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea
previous_output_index1=2
previous_output_amount1=499550

## 2. Sign first input and output with `single|anyonecanpay`

### A. Create first  P2PKH output script.

* Please send testnet coins to the following address: `n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw`

In [ ]:
# bx decode-address n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw
bx address-decode n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw

In [ ]:
# bx script-encode "DUP HASH160 [public key hash] EQUALVERIFY CHECKSIG"

# output_script0=

output_script0=$(bx script-encode "DUP HASH160 ["e48199d47742b245464b1366d95ef26aa4c8bb2c"] EQUALVERIFY CHECKSIG")
echo $output_script0

### B. Set output amount equal to UTXO value.
* Fees will be later be deducted from second output at index 1.

In [ ]:
# Output amount total.

# output_amount0=

output_amount0=$previous_output_amount0
echo $output_amount0


### C. Construct transaction template for signing input and outputs at index 0.

* `sequence: 0xffffffff(hex)/4294967295(dec)`

In [ ]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

#my_tx0=

my_tx0=$(bx tx-encode \
--input $previous_txid0:$previous_output_index0:4294967295 \
--output $output_script0:$output_amount0)

echo $my_tx0

### E. Fetch previous output script which first input is spending

In [ ]:
# bx fetch-tx --format json [previous tx id] 

#previous_output_script0=

previous_output_script0=$(bx fetch-tx --format json $previous_txid0 \
| jq -r ".transaction.outputs[1].script")

echo $previous_output_script0

### F. Sign transaction with `single|anyonecanpay`

In [ ]:
# bx input-sign --sign_type [all|none|single] (--anyone) [private key] "previous output script" [transaction template]

#signature0=

signature0=$(bx input-sign --sign_type single --anyone --index 0 $privatekey_44h_1h_1h_0_1 "$previous_output_script0" $my_tx0)
echo $signature0


## 3. Sign second input with `none|anyonecanpay`


### A. Construct transaction template with both inputs and the first output.

In [ ]:
#my_tx1=

my_tx1=$(bx tx-encode \
--input $previous_txid0:$previous_output_index0:4294967295 \
--input $previous_txid1:$previous_output_index1:4294967295 \
--output $output_script0:$output_amount0)

echo $my_tx1

### B. Fetch previous output script which second input is spending.

In [ ]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script1=

previous_output_script1=$(bx fetch-tx --format json $previous_txid0 \
| jq -r ".transaction.outputs[2].script")

echo $previous_output_script1


In [ ]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature1=

signature1=$(bx input-sign --sign_type none --anyone --index 1 $privatekey_44h_1h_1h_0_2 "$previous_output_script1" $my_tx1)
echo $signature1

# 4. Finalise transaction.

### A. Create second P2PKH output script

* Please send testnet coins to the following address: `n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw`

In [ ]:
# bx script-encode "DUP HASH160 [public key hash] EQUALVERIFY CHECKSIG"

# output_script1=

output_script1=$(bx script-encode "DUP HASH160 ["e48199d47742b245464b1366d95ef26aa4c8bb2c"] EQUALVERIFY CHECKSIG")
echo $output_script1


### B. Compute second output amount.
* Fees will be subtracted from 2nd output.

In [ ]:
# tx bytes: 4 + 1 + 2*(32+4+1+1+72+1+33+4) + 1 + 2*(8+1+25) + 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 2 \* 148 + 1 + 2 \* 34 + 4)
echo $tx_byte_count


In [ ]:
# Second output amount total.

# output_amount1=

output_amount1=$(expr $previous_output_amount1 - 2 \* 400 - $tx_byte_count \* 1)
echo $output_amount1


### C. Construct final transaction
* First re-encode transaction with all in and outputs.

In [ ]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

#my_tx_final=

my_tx_final=$(bx tx-encode \
--input $previous_txid0:$previous_output_index0:4294967295 \
--input $previous_txid1:$previous_output_index1:4294967295 \
--output $output_script0:$output_amount0 \
--output $output_script1:$output_amount1)


* Set input scripts into the final transaction.

In [ ]:
# bx input-set --index 0 "[signature] [public key point]" [transaction template]
# bx input-set --index 1 "[signature] [public key point]" [transaction template]

# my_tx_final=

my_tx_final=$(bx input-set --index 0 "[$signature0] [$publickey_44h_1h_1h_0_1]" $my_tx_final)
my_tx_final=$(bx input-set --index 1 "[$signature1] [$publickey_44h_1h_1h_0_2]" $my_tx_final)


In [ ]:
# Validate.
bx validate-tx $my_tx_final
echo $my_tx_final